### Using snscrape and facebook_scraper

#### install facebookscraper and snscrape libraries

In [ ]:
import csv
import facebook_scraper
import subprocess

class CommentScraper:
    def __init__(self):
        # Set the output file
        self.OUTPUT_FILE = "abusive_comments.csv"

    def scrape_instagram_comments(self, account):
        """Scrape the comments from an Instagram account"""

        # Use the snscrape tool to scrape the comments from the target account
        result = subprocess.run(
            ["snscrape", "instagram-user", account, "--comments"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
        )

        # Open the output file in append mode
        with open(self.OUTPUT_FILE, "a", newline="") as csvfile:
            # Create a CSV writer
            writer = csv.writer(csvfile, delimiter=",")

            # Parse the output of the snscrape tool
            for line in result.stdout.split("\n"):
                # Skip empty lines
                if not line:
                    continue

                # Split the line into columns
                post_id, comment = line.split("\t")

                # Check if the comment is abusive
                if "abusive" in comment.lower():
                    # Write each abusive comment to a row in the CSV file
                    writer.writerow([account, post_id, comment])

    def scrape_facebook_comments(self, account):
        """Scrape the comments from a Facebook account"""

        # Use the facebook-scraper library to scrape the posts from the target account
        posts = facebook_scraper.get_posts(account, pages=1)

        # Open the output file in append mode
        with open(self.OUTPUT_FILE, "a", newline="") as csvfile:
            # Create a CSV writer
            writer = csv.writer(csvfile, delimiter=",")

            # Iterate through the posts and extract the abusive comments
            for post in posts:
                comments = post["comments"]
                for comment in comments:
                    if "abusive" in comment["text"].lower():
                        # Write each abusive comment to a row in the CSV file
                        writer.writerow([account, post["post_id"], comment["text"]])
# Create an instance of the CommentScraper class
scraper = CommentScraper()

# Scrape the comments from each target account
for account in ACCOUNTS:
    if account.startswith("instagram"):
        scraper.scrape_instagram_comments(account)
    elif account.startswith("facebook"):
        scraper.scrape_facebook_comments(account)


# One Downside with this approach is that we cannot use it for multiple users. It is just like hard coding.

#### One more approach I think we can try

### Facebook GraphAPI and Instagram API

#### I will figure out facebook graph api and instagram api as we need to scrape data from every users.

In [ ]:
# Import necessary libraries
import csv
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import facebook
import instagram

# Set up SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

# Define a list of abusive words to check for
abusive_words = ['hate', 'bigot', 'ignorant', 'stupid', 'idiot']

# Authenticate with Facebook and Instagram API
fb_access_token = "XXX"
fb_api = facebook.GraphAPI(access_token=fb_access_token)
insta_client_id = "XXX"
insta_client_secret = "XXX"
insta_access_token = "XXX"
insta_api = instagram.client(client_id=insta_client_id, client_secret=insta_client_secret, access_token=insta_access_token)

# Get comments from Facebook and Instagram
fb_comments = fb_api.get_object("me/comments")
insta_comments = insta_api.media_comments("media-id")

# Open a CSV file for writing
with open("abusive_comments.csv", "w") as csv_file:
  writer = csv.writer(csv_file)

  # Check each comment for abusive words
  for comment in fb_comments + insta_comments:
    scores = analyzer.polarity_scores(comment)
    for word in comment.split():
      if word in abusive_words:
        # Write the abusive comment to the CSV file
        writer.writerow([comment])
        break
